# Code for Robust Predictive Motion Planning by Learning Obstacle Uncertainty

## Import packages

In [1]:
import casadi
import random
import numpy as np
from pytope import Polytope
import matplotlib
import matplotlib.pyplot as plt
import time
import scipy.io as scio
import scipy.stats as stats
import matplotlib.animation as animation
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)

from Planner_D import Planner_D
from Planner_N import Planner_N
from Planner_R import Planner_R

from ModelingSVTrue import ModelingSVTrue

from shapely.geometry import Polygon, Point

%matplotlib auto

Using matplotlib backend: <object object at 0x7f8958577710>


## Define the function for learning the initial control set of obstacle

In [2]:
class Initialization_Set_Estimation( ):
    def __init__(self, Params):
        
        self.N_Sam = Params['N_pre_sam']
        self.H = Params['H']
        self.h = Params['h']
        self.nv = Params['nv']
        self.nu = Params['nu']
        self.LinearProgramming = self.LinearProgramming( )
    
    def Return(self, samples, U_SV_Poly):
        
        theta_initial, y_initial = self.LinearProgramming(samples)
        fW = Polytope(self.H, theta_initial)
        U_hat_initial = fW + y_initial
        
        return theta_initial, y_initial, U_hat_initial
    
    def LinearProgramming(self): 
        H = self.H
        h = self.h
        N_Sam = self.N_Sam
        nv = self.nv
        nu = self.nu
        
        opti = casadi.Opti( )
        
        rho = opti.variable( )
        theta = opti.variable(nv, 1)
        y = opti.variable(nu, 1)
        
        samples = opti.parameter(self.nu, N_Sam)
        
        opti.minimize(rho + np.ones((1, nv))@theta)
        
        for i in range(N_Sam):
            opti.subject_to(-H @ y <= theta - H @ samples[:, i])

        opti.subject_to(H @ y <= 1 - rho)
        opti.subject_to(opti.bounded(0, theta, 1))
        opti.subject_to(opti.bounded(0, rho, 1))
        opti.subject_to(theta <= rho)
            
        opts = {"ipopt.print_level": 0, "print_time": False}
        opti.solver('ipopt', opts)
        
        return opti.to_function('f', [samples], [theta, y])

## Basic Parameters

In [3]:
T = 0.25 # time interval
N = 10   # prediction horizon
infinity  = 1000000 # infinity
max_speed = 1.5     # max speed, m/s
l_veh_sv  = 0.36  
w_veh_sv  = 0.23  
l_veh_ev  = 0.26  
w_veh_ev  = 0.25  
DEV  = 5 # dimenstion of EV
DSV  = 4 # dimenstion of SV
x_low  = 0 + l_veh_sv/2  # min x position, m
y_low  = 0 + l_veh_sv/2  # min y position, m
x_up   = 8 - l_veh_sv/2  # max x position, m
y_up   = 8 - l_veh_sv/2  # max y position, m
Drive_area        = Polytope(np.array([[x_low, y_low], [x_up, y_low], [x_up, y_up], [x_low, y_up]])) 
A_road = Drive_area.A
b_road = Drive_area.b 

## Define initial and target states of both the EV and SV

In [4]:
RefPos_EV = [7, 5.5, 0]
RefPos_SV = [1, 6.75, np.pi] 
x_EV_initial = np.array([0.2, 0.2, 0, 0, 0]) 
x_SV_initial = np.array([6.25, 1.2, -0.25*np.pi, 0])

## EV and SV Information

In [5]:
A_SV = np.array([[1, T, 0, 0], [0, 1, 0, 0], [0, 0, 1, T], [0, 0, 0, 1]])
B_SV = np.array([[0.5*T**2, 0], [T, 0], [0, 0.5*T**2], [0, T]])
low_bound_control  = (-1, -1)   # lower bound on ax ay
up_bound_control   = (1,   1)   # upper bound on ax ay

X_SV_Poly = Polytope(lb = (x_low, 0, y_low, 0),   ub = (x_up, max_speed, y_up, max_speed)) # state admissible set of SV
U_SV_Poly = Polytope(lb = low_bound_control, ub = up_bound_control) # control admissible set of SV

H = U_SV_Poly.A
h = U_SV_Poly.b
nv = np.size(H, 0)
nu = np.size(H, 1)
for i in range(nv):
    H[i, :] = H[i, :]/h[i]
    h[i] = 1
d_min    = np.sqrt((l_veh_sv/2)**2 + (w_veh_sv/2)**2) + np.sqrt((l_veh_ev/2)**2 + (w_veh_ev/2)**2)
d_margin = 0.01

In [6]:
N_Car = 2
opts_Parameters_EV = { 
    'd_min': d_min,
    'T': T,     
    'N': N,
    'l_veh': l_veh_ev,
    'w_veh': w_veh_ev,
    'l_f': 0.08,
    'l_r': 0.08,
    'DEV': DEV,
    'N_Car':N_Car,
    'H': H,
    'h': h,
    'nv': nv,
    'nu': nu,
    'A_SV': A_SV,
    'B_SV': B_SV,
    'U_SV_Poly': U_SV_Poly,
    'infinity': infinity,
    'max_speed': max_speed,
    'd_min': d_min,
    'd_margin': d_margin,
    'Q1': 1, # steering wheel angle
    'Q2': 1, # longitudinal jerk
    'Q3': 1, # long. velocity error
    'Q4': 5, # long. pos. error
    'Q5': 5, # lat. pos. error
    'Q6': 2, # heading. error
    'Q7': 300, # slack variable
    'A_road': A_road,
    'b_road': b_road,
    'v_low': 0, # lower bound of vel.
    'v_up': max_speed, # up bound of vel.
    'acc_low': -0.5,  # lower bound of acc.
    'acc_up': 0.5,     # up bound of acc.
    'delta_low': -0.3, # lower bound of front tire angle.
    'delta_up': 0.3, # up bound of front tire angle
    'RefSpeed': 0, # terminal reference speed of EV
    'RefPos': RefPos_EV, # terminal reference x-y position of EV
}

In [7]:
opts_Parameters_SVTrue = { 
    'T': T,     
    'N': N,
    'l_veh': l_veh_sv,
    'w_veh': w_veh_sv,
    'l_f': 0.14,
    'l_r': 0.14,
    'DSV': 4,
    'Q1': 1, # steering wheel angle
    'Q2': 1, # longitudinal acceleration
    'Q3': 4, # heading. velocity error
    'Q4': 4, # long. pos. error
    'Q5': 4, # lat. pos. error
    'Q6': 4, # velocity error
    'A_road': A_road,
    'b_road': b_road,
    'v_low': 0, # lower bound of vel.
    'v_up': max_speed, # up bound of vel.
    'acc_low': -0.3, #-3.43, # lower bound of acc.
    'acc_up': 0.3,#2.27, # up bound of acc.
    'delta_low': -0.6, # lowver bound of front tire angle.
    'delta_up': 0.6, # up bound of front tire angle
}

## Motion Planning

In [8]:
MotionPlanner_D =  Planner_D(Params     = opts_Parameters_EV)
MotionPlanner_N =  Planner_N(Params     = opts_Parameters_EV)
MotionPlanner_R =  Planner_R(Params     = opts_Parameters_EV)
ModelingSVTrue  = ModelingSVTrue(Params = opts_Parameters_SVTrue)

In [9]:
N_pre_sam  = 8
min_s      = 0.01
samples_SV = np.array([[-min_s, -min_s], [-min_s, 0], [-min_s, min_s], [0, min_s], [min_s, min_s], [min_s, 0], [min_s, -min_s], [0, -min_s]])
samples_SV = samples_SV.reshape(2, N_pre_sam)
opts_Parameters_Initial_Set_Estimation ={'N_pre_sam': N_pre_sam,'H': H, 'h': h,'nv': nv,'nu': nu}
Initialization_Set_Estimation = Initialization_Set_Estimation(Params = opts_Parameters_Initial_Set_Estimation)
theta_initial, y_initial, U_hat_initial = Initialization_Set_Estimation.Return(samples_SV, U_SV_Poly)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [10]:
K_N = 55
# SV
State_SV     = np.zeros((4, K_N + 1))
Control_SV =  np.zeros((2, K_N))
# EV: state
State_EV_D     = np.zeros((DEV, K_N + 1))
State_EV_N      = np.zeros((DEV, K_N + 1))
State_EV_R     = np.zeros((DEV, K_N + 1))
# EV: control action
Control_EV_D   = np.zeros((2, K_N))
Control_EV_N    = np.zeros((2, K_N))
Control_EV_R   = np.zeros((2, K_N))
# EV: cost
J_D = np.array([None]*(K_N))
J_N  = np.array([None]*(K_N))
J_R = np.array([None]*(K_N))
# SV & EV planning
X_SV_Planning = list( ) # SV planning full-state trajectory in the horizon
X_EV_Planning_D = list( )
X_EV_Planning_N = list( )
X_EV_Planning_R = list( )
# EV: robust tube
Aug_Occupancy_SV_D = list( ) # augmented description of occupancy of SV over prediction horizon under D prediciion
Aug_Occupancy_SV_N = list( ) # augmented description of occupancy of SV over prediction horizon under risk-aware prediciion
Aug_Occupancy_SV_R = list( )  # augmented description of occupancy of SV over prediction horizon under nominal robust prediciion
# EV: set parameters for online iteration of the uncertainty quantification
Theta = list( ) # save the value of alpha_k at very time step
Y     = list( )   # save the value of rho_k at every time step
U_Hat = list( ) # save the polytope \hat{U}_k at every time step
# initiate
State_SV[:, 0] = x_SV_initial
State_EV_D[:, 0] = x_EV_initial
State_EV_N[:, 0] = x_EV_initial
State_EV_R[:, 0] = x_EV_initial
Theta.append(theta_initial)
Y.append(y_initial)
U_Hat.append(U_hat_initial)

In [11]:
start = time.perf_counter( )

for k in range(K_N):
    if k == 0:
        u_sv_before = samples_SV[:, -1]
    else:
        u_sv_before = np.array([Control_SV[:, k-1][1]*np.cos(State_SV[:, k-1][2]), Control_SV[:, k-1][1]*np.sin(State_SV[:, k-1][2])])
    print('the step is', k)
    #print('u_sv_before is', u_sv_before)
    control_SV_k, x_SV_planning_k = ModelingSVTrue.Return(State_SV[:, k], RefPos_SV)
    U_k_D, trajectory_planning_k_D, J_D_k, Occupancy_SV_aug_D_k = MotionPlanner_D.Return(State_SV[:, k],  State_EV_D[:, k])
    U_k_N, trajectory_planning_k_N, J_N_k, Occupancy_SV_aug_N_k, theta_k, y_k, U_hat_k = MotionPlanner_N.Return(State_SV[:, k], State_EV_N[:, k], Theta[k-1], Y[k-1], u_sv_before)
    U_k_R, trajectory_planning_k_R, J_R_k, Occupancy_SV_aug_R_k = MotionPlanner_R.Return(State_SV[:, k], State_EV_R[:, k])
   
    Control_SV[:, k] = control_SV_k
    X_SV_Planning.append(x_SV_planning_k)
    State_SV[:, k + 1] = x_SV_planning_k[:, 1]
    
    X_EV_Planning_D.append(trajectory_planning_k_D)
    State_EV_D[:, k+1] = trajectory_planning_k_D[:, 1]
    Control_EV_D[:, k] = U_k_D
    Aug_Occupancy_SV_D.append(Occupancy_SV_aug_D_k)

    X_EV_Planning_N.append(trajectory_planning_k_N)
    State_EV_N[:, k+1] = trajectory_planning_k_N[:, 1]
    Control_EV_N[:, k] = U_k_N
    Aug_Occupancy_SV_N.append(Occupancy_SV_aug_N_k)
    Theta.append(theta_k)
    Y.append(y_k)
    U_Hat.append(U_hat_k)

    X_EV_Planning_R.append(trajectory_planning_k_R)
    State_EV_R[:, k + 1] = trajectory_planning_k_R[:, 1]
    Control_EV_R[:, k] = U_k_R
    Aug_Occupancy_SV_R.append(Occupancy_SV_aug_R_k)
    
    J_D[k] = J_D_k
    J_N[k] = J_N_k
    J_R[k] = J_R_k
    
    samples_SV = np.hstack((samples_SV, np.array([Control_SV[:, k][1]*np.cos(State_SV[:, k][2]), Control_SV[:, k][1]*np.sin(State_SV[:, k][2])]).reshape(2, 1)))
    
end = time.perf_counter( )
total_t = end - start
print ('Average excution time: ', total_t/k)

t = np.arange(0, T*(K_N + 1), T, dtype=float) # total simulation time

the step is 0
the step is 1
the step is 2
the step is 3
the step is 4
the step is 5
the step is 6
the step is 7
the step is 8
the step is 9
the step is 10
the step is 11
the step is 12
the step is 13
the step is 14
the step is 15
the step is 16
the step is 17
the step is 18
the step is 19
the step is 20
the step is 21
the step is 22
the step is 23
the step is 24
the step is 25
the step is 26
the step is 27
the step is 28
the step is 29
the step is 30
the step is 31
the step is 32
the step is 33
the step is 34
the step is 35
the step is 36
the step is 37
the step is 38
the step is 39
the step is 40
the step is 41
the step is 42
the step is 43
the step is 44
the step is 45
the step is 46
the step is 47
the step is 48
the step is 49
the step is 50
the step is 51
the step is 52
the step is 53
the step is 54
Average excution time:  0.33599567272221975
